In [1]:
# Choisir le Kernel PySpark sur JupyterHub de L'instance EMR puis éxécuter la cellule elle lancera une session Spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1683575663832_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1683575663832_0002,pyspark,idle,Link,Link,None,✔


## Imports (installer via le fichier bootstrap lors de la création du cluster EMR)

In [3]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox, TextArea
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from pyspark.ml.feature import PCA, VectorAssembler

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Définitions de chemin des dossiers via S3

In [4]:
PATH = 's3://p8datalubinstephane'
PATH_Data = PATH+'/Training'
PATH_Result = PATH+'/Results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://p8datalubinstephane
PATH_Data:   s3://p8datalubinstephane/Training
PATH_Result: s3://p8datalubinstephane/Results

### Lecture des images

In [5]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://p8datalubins...|2023-04-14 11:35:32|  7437|[FF D8 FF E0 00 1...|
|s3://p8datalubins...|2023-04-14 11:35:33|  7434|[FF D8 FF E0 00 1...|
|s3://p8datalubins...|2023-04-14 09:13:58|  7424|[FF D8 FF E0 00 1...|
|s3://p8datalubins...|2023-04-14 11:35:41|  7423|[FF D8 FF E0 00 1...|
|s3://p8datalubins...|2023-04-14 11:35:49|  7416|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

In [7]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+------------------------------------------------------------+--------------+
|path                                                        |label         |
+------------------------------------------------------------+--------------+
|s3://p8datalubinstephane/Training/Raspberry/176_100.jpg     |Raspberry     |
|s3://p8datalubinstephane/Training/Raspberry/179_100.jpg     |Raspberry     |
|s3://p8datalubinstephane/Training/Pineapple Mini/170_100.jpg|Pineapple Mini|
|s3://p8datalubinstephane/Training/Raspberry/157_100.jpg     |Raspberry     |
|s3://p8datalubinstephane/Training/Raspberry/131_100.jpg     |Raspberry     |
+------------------------------------------------------------+--------------+
only showing top 5 rows

None

### Création du modèle à partir de MobileNetV2

In [8]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

## Fonctions communes

In [12]:
def model_fn():
    """
    Retourne le modèle MobileNetV2 et diffuse les poids.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
def preprocess(content):
    """
    Redimensionne les images.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurisation d'une série d'images brutes à l'aide du modèle.    
    renvoie une pd.Series de caractéristiques d'images
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # Pour certaines couches, les caractéristiques de sortie seront des tenseurs multidimensionnels 
    # Nous aplatissons les tenseurs de caractéristiques en vecteurs pour faciliter le stockage dans Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    Cette méthode est un UDF pandas Scalar Iterator enveloppant notre fonction de featurisation. *
    Cette méthode renvoie une colonne Spark DataFrame de type ArrayType(FloatType).

    :param content_series_iter : Cet argument est un itérateur sur les lots de données, 
    où chaque lot est une série pandas de données d'image. ''' 
    # Avec les UDF pandas Scalar Iterator, nous pouvons charger le modèle une fois et ensuite le réutiliser 
    # pour plusieurs lots de données.  Cela permet d'amortir les frais généraux liés au chargement de gros modèles.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/mnt1/yarn/usercache/livy/appcache/application_1683575663832_0002/container_1683575663832_0002_01_000001/pyspark.zip/pyspark/sql/pandas/functions.py:398: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

### Exécutions des actions d'extractions de features

In [14]:
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Conversion des variables pour réduction de dimension PCA

- Conversion de ArrayType vers VectorsUDT

In [16]:
n = 1280
va = VectorAssembler(inputCols=["features[{}]".format(i) for i in range(n)], outputCol = 'vaFeatures')
vafeatures_df = va.transform(features_df.select("*", *[features_df["features"][i] for i in range(n)]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
vafeatures_df.select('vaFeatures')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[vaFeatures: vector]

#### Reduction de dimension

In [18]:
pca = PCA(k=150, inputCol ='vaFeatures', outputCol ='pcaFeatures')
pca_model = pca.fit(vafeatures_df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
#Variance par composantes
pca_model.explainedVariance

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DenseVector([0.0969, 0.0739, 0.0623, 0.0508, 0.0363, 0.0283, 0.0269, 0.0229, 0.0206, 0.0179, 0.0165, 0.0151, 0.0142, 0.0136, 0.0127, 0.0124, 0.0109, 0.0107, 0.0097, 0.009, 0.0088, 0.0086, 0.0078, 0.0077, 0.0074, 0.007, 0.0066, 0.0064, 0.006, 0.0058, 0.0056, 0.0054, 0.0052, 0.0049, 0.0049, 0.0048, 0.0046, 0.0045, 0.0044, 0.0041, 0.0041, 0.0039, 0.0039, 0.0037, 0.0036, 0.0036, 0.0035, 0.0033, 0.0032, 0.0032, 0.0031, 0.003, 0.0029, 0.0028, 0.0027, 0.0027, 0.0026, 0.0025, 0.0025, 0.0024, 0.0024, 0.0024, 0.0023, 0.0023, 0.0022, 0.0022, 0.0021, 0.0021, 0.0021, 0.002, 0.002, 0.002, 0.0019, 0.0019, 0.0018, 0.0018, 0.0018, 0.0018, 0.0017, 0.0017, 0.0017, 0.0016, 0.0016, 0.0016, 0.0016, 0.0016, 0.0015, 0.0015, 0.0015, 0.0014, 0.0014, 0.0014, 0.0014, 0.0013, 0.0013, 0.0013, 0.0013, 0.0013, 0.0013, 0.0013, 0.0012, 0.0012, 0.0012, 0.0012, 0.0012, 0.0012, 0.0012, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.0009, 0.0009, 0.0009, 0.0009, 0

In [20]:
sum(pca_model.explainedVariance)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8619198841794318

features_reduced_df = pca_model.transform(vafeatures_df)

In [21]:
reduced_df = pca_model.transform(vafeatures_df).select("path","label","pcaFeatures")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Ecriture des paquets sur S3

In [22]:
reduced_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Chargement des données enregistrées et validation du résultat

In [23]:
df = pd.read_parquet(PATH_Result, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                path  ...                                        pcaFeatures
0  s3://p8datalubinstephane/Training/Raspberry/18...  ...  {'type': 1, 'size': None, 'indices': None, 'va...
1  s3://p8datalubinstephane/Training/Pineapple Mi...  ...  {'type': 1, 'size': None, 'indices': None, 'va...
2  s3://p8datalubinstephane/Training/Raspberry/16...  ...  {'type': 1, 'size': None, 'indices': None, 'va...
3  s3://p8datalubinstephane/Training/Pineapple Mi...  ...  {'type': 1, 'size': None, 'indices': None, 'va...
4  s3://p8datalubinstephane/Training/Raspberry/41...  ...  {'type': 1, 'size': None, 'indices': None, 'va...

[5 rows x 3 columns]

In [25]:
df.loc[0,'pcaFeatures']['values']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([ 1.70858705e+00,  3.87665141e+00, -2.23998961e+00, -1.21362246e+01,
       -2.33797480e+00,  9.88731411e-02, -1.28771601e-01,  3.33325937e+00,
        9.18320361e+00,  3.97675552e-01,  1.09852718e+01,  2.00125630e+00,
       -1.26899441e-01, -1.88466638e+00, -3.13117067e+00, -4.10947065e-01,
        7.66691925e+00, -2.90944201e+00,  2.98742153e+00, -9.91379304e-01,
       -2.25221948e+00, -3.69266153e-01, -2.02184042e+00, -2.91086370e-01,
       -5.91584931e+00,  9.76708025e-04,  2.06495357e+00,  1.06219344e+00,
        3.44373256e+00,  1.95799187e+00, -8.48734049e-01,  9.82012137e-01,
       -1.50116985e+00,  1.21786340e+00,  6.15524462e-01, -2.48816071e+00,
       -1.51394959e+00,  6.78806602e-01,  2.03835970e+00, -2.54058645e-01,
        2.27269260e+00,  9.11938631e-01,  2.32686179e+00, -2.21566714e-01,
        2.10675813e+00,  6.13371305e-01, -9.24181078e-01, -2.92382092e-01,
        1.47412513e+00, -1.54714665e+00,  9.37628770e-01, -1.71591134e+00,
       -2.22862830e-01, -